# DQN

In [232]:
# import package needed
%matplotlib inline
import matplotlib.pyplot as plt
import os
os.environ["SDL_VIDEODRIVER"] = "dummy"  # this line make pop-out window not appear
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import tensorflow as tf
import numpy as np
import skimage.color
import skimage.transform
from ple.games.flappybird import FlappyBird
from ple import PLE
game = FlappyBird()
env = PLE(game, fps=30, display_screen=False)  # environment interface to game

In [233]:
# define input size
screen_width = 80
screen_height = 80
num_stack = 4

In [234]:
import math
import copy
from collections import defaultdict
MIN_EXPLORING_RATE = 10e-4
state = game.getGameState()
class Agent:

  def __init__(self, name, num_action, t=0, discount_factor=0.99):
    self.exploring_rate = 0.1
    self.discount_factor = discount_factor
    self.num_action = num_action
    self.name = name
    self.bucket_range_per_feature = bucket_range_per_feature
    with tf.variable_scope(name):
        
      self.build_model()
    
  def get_state_idx(self, state):
    # instead of using absolute position of pipe, use relative position
    state = copy.deepcopy(state)
    state['next_next_pipe_bottom_y'] -= state['player_y']
    state['next_next_pipe_top_y'] -= state['player_y']
    state['next_pipe_bottom_y'] -= state['player_y']
    state['next_pipe_top_y'] -= state['player_y']

    # sort to make list converted from dict ordered in alphabet order
    state_key = [k for k, v in sorted(state.items())]

    # do bucketing to decrease state space to speed up training
    state_idx = []
    for key in state_key:
        state_idx.append(state[key] / self.bucket_range_per_feature[key])
    return tuple(state_idx)

  def build_model(self):
    # input: current screen, selected action and reward
    #self.input_screen = tf.placeholder(tf.float32, shape=[None, screen_width, screen_height, num_stack])
    self.input_state = tf.placeholder(tf.float32, shape=[None, 8, num_stack])
    self.action = tf.placeholder(tf.int32, [None])
    self.reward = tf.placeholder(tf.float32, [None])
    self.is_training = tf.placeholder(tf.bool, shape=[])

    def net(state, reuse=False):
      with tf.variable_scope("fc", initializer=tf.truncated_normal_initializer(stddev=1e-2)):
        for idx in range(num_stack):
          if idx == 0:
            pre_dense = tf.layers.dense(inputs=state[:, :, idx], units=6400, activation=tf.nn.relu, reuse=reuse)
            pre_dense = tf.reshape(pre_dense, [-1, 8, 1])
          else:
            pre_stack = tf.layers.dense(inputs=state[:, :, idx], units=6400, activation=tf.nn.relu, reuse=True)
            pre_stack = tf.reshape(pre_stack, [-1, 8, 1])
            pre_dense = tf.concat([pre_dense, pre_stack], 2)
                
      with tf.variable_scope("layers", reuse=reuse, initializer=tf.truncated_normal_initializer(stddev=1e-2)):
        state_in = tf.reshape(pre_dense, [-1, screen_width, screen_height, num_stack]) 
        conv1 = tf.layers.conv2d(inputs=state_in, filters=32, kernel_size=[8, 8], 
                                 strides=[4, 4], padding='SAME', activation=tf.nn.relu)
        pool1 = tf.layers.max_pooling2d(conv1, pool_size=[2, 2], strides=[2, 2], padding='SAME')
        conv2 = tf.layers.conv2d(inputs=pool1, filters=64, kernel_size=[4, 4],
                                 strides=[2, 2], padding='SAME', activation=tf.nn.relu)
        conv3 = tf.layers.conv2d(inputs=conv2, filters=64, kernel_size=[3, 3],
                                 strides=[1, 1], padding='SAME', activation=tf.nn.relu)
        flat = tf.contrib.layers.flatten(conv3)
        dense = tf.layers.dense(inputs=flat, units=512, activation=tf.nn.relu)
        Q = tf.layers.dense(inputs=dense, units=self.num_action, activation=None)
        return Q

    # optimize
    self.output = net(self.input_state)  # Q(s,a,theta) for all a, shape (batch_size, num_action)
    index = tf.stack([tf.range(tf.shape(self.action)[0]), self.action], axis=1)
    self.esti_Q = tf.gather_nd(self.output,index)  # Q(s,a,theta) for selected action, shape (batch_size, 1)
    self.max_Q = tf.reduce_max(self.output, axis=1)  # max(Q(s',a',theta')), shape (batch_size, 1)
    self.tar_Q = tf.placeholder(tf.float32, [None])
    # loss = E[r+max(Q(s',a',theta'))-Q(s,a,theta)]
    self.loss = tf.reduce_mean(tf.square(self.reward + self.discount_factor * self.tar_Q - self.esti_Q))

    optimizer = tf.train.AdamOptimizer(learning_rate=1e-5)
    self.g_gvs = optimizer.compute_gradients(self.loss, var_list=[v for v in tf.global_variables() if self.name in v.name])
    self.train_op = optimizer.apply_gradients(self.g_gvs)
    self.pred = tf.argmax(self.output, axis=1)  # select action with highest action-value, only used in inference

  def select_action(self, input_state, sess):
    # epsilon-greedy
    if np.random.rand() < self.exploring_rate:
      action = np.random.choice(num_action)  # Select a random action
    else:
      input_states = np.array(input_state).transpose([1, 0])
      feed_dict = {
          self.input_state: input_states[None, :],
          self.is_training: False,
      }
      action = sess.run(
          self.pred,
          feed_dict=feed_dict)[0]  # Select the action with the highest q
    return action

  def update_policy(self, input_state, actions, rewards, input_screens_plum,
                    terminal, target_netwrok):
    # use max_Q estimate from target one to update online one
    feed_dict = {
        target_netwrok.input_state:
            np.array(input_screens_plum).transpose([0, 2, 1]),
        target_netwrok.is_training:
            True,
    }
    max_Q = sess.run(target_netwrok.max_Q, feed_dict=feed_dict)
    max_Q *= ~np.array(terminal)
    feed_dict = {
        self.input_state: np.array(input_state).transpose([0, 2, 1]),
        self.tar_Q: max_Q,
        self.action: actions,
        self.reward: rewards,
        self.is_training: True,
    }
    loss, _ = sess.run([self.loss, self.train_op], feed_dict=feed_dict)
    return loss

  def update_parameters(self, episode):
    if self.exploring_rate > MIN_EXPLORING_RATE:
      self.exploring_rate -= (0.1 - MIN_EXPLORING_RATE) / 3000000

  def shutdown_explore(self):
    # make action selection greedy
    self.exploring_rate = 0


In [235]:
def get_update_ops():
  # return operations assign weight to target network
  src_vars = [v for v in tf.global_variables() if 'online' in v.name]
  tar_vars = [v for v in tf.global_variables() if 'target' in v.name]
  update_ops = []
  for src_var, tar_var in zip(src_vars, tar_vars):
    update_ops.append(tar_var.assign(src_var))
  return update_ops


def update_target(update_ops, sess):
  sess.run(update_ops)

In [236]:
# init agent
tf.reset_default_graph()
num_action = len(env.getActionSet())
bucket_range_per_feature = {
  'next_next_pipe_bottom_y': 40,
  'next_next_pipe_dist_to_player': 512,
  'next_next_pipe_top_y': 40,
  'next_pipe_bottom_y': 20,
  'next_pipe_dist_to_player': 20,
  'next_pipe_top_y': 20,
  'player_vel': 4,
  'player_y': 16
}
# agent for frequently updating
online_agent = Agent('online', num_action, bucket_range_per_feature)

# agent for slow updating
target_agent = Agent('target', num_action, bucket_range_per_feature)
update_ops = get_update_ops()

In [237]:
class Replay_buffer():

  def __init__(self, buffer_size=50000):
    self.experiences = []
    self.buffer_size = buffer_size

  def add(self, experience):
    if len(self.experiences) >= self.buffer_size:
      self.experiences.pop(0)
    self.experiences.append(experience)

  def sample(self, size):
    """
        sameple experience from buffer
        """
    if size > len(self.experiences):
      experiences_idx = np.random.choice(len(self.experiences), size=size)
    else:
      experiences_idx = np.random.choice(
          len(self.experiences), size=size, replace=False)
    # from all sampled experiences, extract a tuple of (s,a,r,s')
    states = []
    actions = []
    rewards = []
    screens_plum = []
    terminal = []
    for i in range(size):
      states.append(self.experiences[experiences_idx[i]][0])
      actions.append(self.experiences[experiences_idx[i]][1])
      rewards.append(self.experiences[experiences_idx[i]][2])
      screens_plum.append(self.experiences[experiences_idx[i]][3])
      terminal.append(self.experiences[experiences_idx[i]][4])
    return states, actions, rewards, screens_plum, terminal

In [238]:
# init buffer
buffer = Replay_buffer()

In [239]:
def make_anim(images, fps=60, true_image=False):
  duration = len(images) / fps
  import moviepy.editor as mpy

  def make_frame(t):
    try:
      x = images[int(len(images) / duration * t)]
    except:
      x = images[-1]

    if true_image:
      return x.astype(np.uint8)
    else:
      return ((x + 1) / 2 * 255).astype(np.uint8)

  clip = mpy.VideoClip(make_frame, duration=duration)
  clip.fps = fps
  return clip

In [240]:
# init all
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=config)
sess.run(tf.global_variables_initializer())

In [244]:
from IPython.display import Image, display

update_every_t_step = 3
print_every_episode = 10
save_video_every_episode = 1000
NUM_EPISODE = 30000
NUM_EXPLORE = 20

# we can redefine origin reward function
reward_values = {
    "positive": 1,  # reward pass a pipe
    "tick": 0.1,  # reward per timestamp
    "loss": -1,  # reward of gameover
}
for episode in range(0, NUM_EPISODE + 1):

  # Reset the environment
  game = FlappyBird()
  # for demo purpose, the following code is trained in the same scene,
  env = PLE(
      game,
      fps=30,
      display_screen=False,
      reward_values=reward_values,
      rng=np.random.RandomState(1))
  env.reset_game()
  env.act(0)  # dummy input to make sure input screen is correct

  # record frame
  if episode % save_video_every_episode == 0:
    frames = [env.getScreenRGB()]

  # for every 500 episodes, shutdown exploration to see performance of greedy action
  if episode % print_every_episode == 0:
    online_agent.shutdown_explore()

  # grayscale input screen for this episode
  #input_screens = [preprocess(env.getScreenGrayscale())] * 4
  input_state = [online_agent.get_state_idx(game.getGameState())] * 4
    
  # experience for this episode, store all (s,a,r,s') tuple
  experience = []

  # cumulate reward for this episode
  cum_reward = 0

  t = 0
  while not env.game_over():
    #print(np.array(input_state).shape)
    # feed four previous screen, select an action
    action = online_agent.select_action(input_state[-4:], sess)

    # execute the action and get reward
    reward = env.act(env.getActionSet()[action])

    # record frame
    if episode % save_video_every_episode == 0:
      frames.append(env.getScreenRGB())

    # cumulate reward
    cum_reward += reward

    # append grayscale screen for this episode
    #input_screens.append(preprocess(env.getScreenGrayscale()))
    input_state.append(online_agent.get_state_idx(game.getGameState()))
    # append experience for this episode
    buffer.add((input_state[-5:-1], action, reward, input_state[-4:],
                env.game_over()))
    t += 1

    # update agent
  if episode > NUM_EXPLORE:
    train_states, train_actions, train_rewards, train_states_plum, terminal = buffer.sample(
        32)
    loss = online_agent.update_policy(train_states, train_actions,
                                      train_rewards, train_states_plum,
                                      terminal, target_agent)
  if t % update_every_t_step == 0 and episode > NUM_EXPLORE:
    update_target(update_ops, sess)

  # update explore rating and learning rate
  online_agent.update_parameters(episode)
  target_agent.update_parameters(episode)

  if episode % print_every_episode == 0 and episode > NUM_EXPLORE:
    print(
        "[{}] time live:{}, cumulated reward: {}, exploring rate: {}, loss: {}".
        format(episode, t, cum_reward, target_agent.exploring_rate, loss))

  if episode % save_video_every_episode == 0:  # for every 100 episode, record an animation
    clip = make_anim(frames, fps=60, true_image=True).rotate(-90)
    clip.write_videofile("movie/DQN-{}.webm".format(episode), fps=60)

[MoviePy] >>>> Building video movie/DQN-0.webm
[MoviePy] Writing video movie/DQN-0.webm


100%|██████████| 63/63 [00:00<00:00, 89.73it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: movie/DQN-0.webm 



[30] time live:61, cumulated reward: 5.099999999999994, exploring rate: 0.0998999440000106, loss: 0.0014970132615417242
[40] time live:61, cumulated reward: 5.099999999999994, exploring rate: 0.09989961400001063, loss: 0.05540362000465393
[50] time live:61, cumulated reward: 5.099999999999994, exploring rate: 0.09989928400001066, loss: 0.027302715927362442
[60] time live:61, cumulated reward: 5.099999999999994, exploring rate: 0.0998989540000107, loss: 0.0279642753303051
[70] time live:61, cumulated reward: 5.099999999999994, exploring rate: 0.09989862400001073, loss: 0.055154576897621155
[80] time live:61, cumulated reward: 5.099999999999994, exploring rate: 0.09989829400001077, loss: 0.0004940094659104943
[90] time live:61, cumulated reward: 5.099999999999994, exploring rate: 0.0998979640000108, loss: 0.00044269702630117536
[100] time live:61, cumulated reward: 5.099999999999994, exploring rate: 0.09989763400001084, loss: 0.027507835999131203
[110] time live:61, cumulated reward: 5.0

[710] time live:61, cumulated reward: 5.099999999999994, exploring rate: 0.09987750400001297, loss: 0.021696938201785088
[720] time live:61, cumulated reward: 5.099999999999994, exploring rate: 0.099877174000013, loss: 0.000530610210262239
[730] time live:61, cumulated reward: 5.099999999999994, exploring rate: 0.09987684400001304, loss: 0.0005528137553483248
[740] time live:61, cumulated reward: 5.099999999999994, exploring rate: 0.09987651400001307, loss: 0.02068259008228779
[750] time live:61, cumulated reward: 5.099999999999994, exploring rate: 0.09987618400001311, loss: 0.0009218280902132392
[760] time live:61, cumulated reward: 5.099999999999994, exploring rate: 0.09987585400001314, loss: 0.04127369076013565
[770] time live:61, cumulated reward: 5.099999999999994, exploring rate: 0.09987552400001318, loss: 0.08157572150230408
[780] time live:61, cumulated reward: 5.099999999999994, exploring rate: 0.09987519400001321, loss: 0.04060322046279907
[790] time live:61, cumulated reward

...

[MoviePy] Done.
[MoviePy] >>>> Video ready: movie/DQN-29000.webm 



[29010] time live:64, cumulated reward: 5.399999999999993, exploring rate: 0.09894360400011179, loss: 0.19456252455711365
[29020] time live:74, cumulated reward: 7.39999999999999, exploring rate: 0.09894327400011182, loss: 0.047723181545734406
[29030] time live:73, cumulated reward: 7.29999999999999, exploring rate: 0.09894294400011186, loss: 0.1134653314948082
[29040] time live:73, cumulated reward: 7.29999999999999, exploring rate: 0.0989426140001119, loss: 0.07905898988246918
[29050] time live:74, cumulated reward: 7.39999999999999, exploring rate: 0.09894228400011193, loss: 0.04939939081668854
[29060] time live:74, cumulated reward: 7.39999999999999, exploring rate: 0.09894195400011196, loss: 0.06536512821912766
[29070] time live:66, cumulated reward: 6.5999999999999925, exploring rate: 0.098941624000112, loss: 0.06542250514030457
[29080] time live:97, cumulated reward: 9.699999999999982, exploring rate: 0.09894129400011203, loss: 0.02556007355451584
[29090] time live:61, cumulated

[29680] time live:184, cumulated reward: 21.400000000000034, exploring rate: 0.09892149400011413, loss: 0.11246661096811295
[29690] time live:140, cumulated reward: 15.999999999999972, exploring rate: 0.09892116400011416, loss: 0.1630907505750656
[29700] time live:73, cumulated reward: 7.29999999999999, exploring rate: 0.0989208340001142, loss: 0.03218374773859978
[29710] time live:72, cumulated reward: 7.19999999999999, exploring rate: 0.09892050400011423, loss: 0.048642948269844055
[29720] time live:75, cumulated reward: 7.499999999999989, exploring rate: 0.09892017400011427, loss: 0.2433781921863556
[29730] time live:138, cumulated reward: 15.799999999999969, exploring rate: 0.0989198440001143, loss: 0.131490096449852
[29740] time live:136, cumulated reward: 14.599999999999968, exploring rate: 0.09891951400011434, loss: 0.028123706579208374
[29750] time live:134, cumulated reward: 14.399999999999968, exploring rate: 0.09891918400011437, loss: 0.07230013608932495
[29760] time live:13

100%|██████████| 140/140 [00:01<00:00, 90.79it/s] 

[MoviePy] Done.
[MoviePy] >>>> Video ready: movie/DQN-30000.webm 



#### 最好的結果

In [247]:
from moviepy.editor import *
clip = VideoFileClip("movie/DQN-30000.webm")
display(clip.ipython_display(fps=60, autoplay=1, loop=1))

100%|██████████| 140/140 [00:00<00:00, 644.19it/s]


#### 進入第一個障礙物

In [280]:
from moviepy.editor import *
clip = VideoFileClip("movie/DQN-11000.webm")
display(clip.ipython_display(fps=60, autoplay=1, loop=1))

100%|██████████| 68/68 [00:00<00:00, 1048.46it/s]


## Policy Gradient

In [262]:
import math
import copy
from collections import defaultdict
MIN_EXPLORING_RATE = 0.01
MIN_LEARNING_RATE = 0.1


class Policy_Gradiebt_Agent:

  def __init__(self, name, num_action, t=0, discount_factor=0.99):
    self.discount_factor = discount_factor
    self.num_action = num_action
    self.name = name
    self.bucket_range_per_feature = bucket_range_per_feature
    with tf.variable_scope(name):
      self.build_model()
    
  def get_state_idx(self, state):
    # instead of using absolute position of pipe, use relative position
    state = copy.deepcopy(state)
    state['next_next_pipe_bottom_y'] -= state['player_y']
    state['next_next_pipe_top_y'] -= state['player_y']
    state['next_pipe_bottom_y'] -= state['player_y']
    state['next_pipe_top_y'] -= state['player_y']

    # sort to make list converted from dict ordered in alphabet order
    state_key = [k for k, v in sorted(state.items())]

    # do bucketing to decrease state space to speed up training
    state_idx = []
    for key in state_key:
        state_idx.append(state[key] / self.bucket_range_per_feature[key])
    return tuple(state_idx)

  def build_model(self):
    # input: current screen, selected action and reward
    self.input_state = tf.placeholder(
        tf.float32, shape=[None, 8, num_stack])
    self.action = tf.placeholder(tf.int32, [None])
    self.reward = tf.placeholder(tf.float32, [None])
    self.is_training = tf.placeholder(tf.bool, shape=[])

    def net(state, reuse=False):
      with tf.variable_scope("fc"):
        for idx in range(num_stack):
          if idx == 0:
            pre_dense = tf.layers.dense(inputs=state[:, :, idx], units=6400, activation=tf.nn.relu, reuse=reuse)
            pre_dense = tf.reshape(pre_dense, [-1, 8, 1])
          else:
            pre_stack = tf.layers.dense(inputs=state[:, :, idx], units=6400, activation=tf.nn.relu, reuse=True)
            pre_stack = tf.reshape(pre_stack, [-1, 8, 1])
            pre_dense = tf.concat([pre_dense, pre_stack], 2)
      with tf.variable_scope("layers", reuse=reuse):
        state_in = tf.reshape(pre_dense, [-1, screen_width, screen_height, num_stack]) 
        conv1 = tf.layers.conv2d(
            inputs=state_in,
            filters=32,
            kernel_size=[8, 8],
            strides=[4, 4],
            padding='SAME',
            activation=tf.nn.relu)
        pool1 = tf.layers.max_pooling2d(
            conv1, pool_size=[2, 2], strides=[2, 2], padding='SAME')

        conv2 = tf.layers.conv2d(
            inputs=pool1,
            filters=64,
            kernel_size=[4, 4],
            strides=[2, 2],
            padding='SAME',
            activation=tf.nn.relu)
        conv3 = tf.layers.conv2d(
            inputs=conv2,
            filters=64,
            kernel_size=[3, 3],
            strides=[1, 1],
            padding='SAME',
            activation=tf.nn.relu)
        self.flat = tf.contrib.layers.flatten(conv3)

        self.dense1 = tf.layers.dense(
            inputs=self.flat, units=512, activation=tf.nn.relu)
        self.dense2 = tf.layers.dense(
            inputs=self.dense1, units=self.num_action, activation=None)
        return self.dense2

    # optimize
    self.output_logit = net(
        self.input_state
    )  # logit of probility(P(s,a,theta)) for all a, shape (batch_size, num_action)
    index = tf.stack([tf.range(tf.shape(self.action)[0]), self.action], axis=1)
    self.prob = tf.gather_nd(
        tf.nn.softmax(self.output_logit),
        index)  # P(s,a,theta) for selected action, shape (batch_size, 1)

    # loss = E[log(p(s,a))*r]
    # because we want to maximize objective, add negative sign before loss
    self.loss = -tf.reduce_mean(tf.log(self.prob + 0.00000001) * self.reward)
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-6)
    g_gvs = optimizer.compute_gradients(
        self.loss,
        var_list=[v for v in tf.global_variables() if self.name in v.name])
    self.train_op = optimizer.apply_gradients(g_gvs)

    self.pred = tf.multinomial(self.output_logit,
                               1)  # sample action from distribution

  def select_action(self, input_state, sess):
    input_states = np.array(input_state).transpose([1, 0])
    feed_dict = {
        self.input_state: input_states[None, :],
        self.is_training: False,
    }
    action = sess.run(
        self.pred,
        feed_dict=feed_dict)[0][0]  # sameple action from distribution
    return action

  def update_policy(self, input_state, actions, rewards, input_states_plum):
    feed_dict = {
        self.input_state: np.array(input_state).transpose([0, 2, 1]),
        self.action: actions,
        self.reward: rewards,
        self.is_training: True,
    }
    loss, _ = sess.run([self.loss, self.train_op], feed_dict=feed_dict)
    return loss

In [263]:
# init agent
tf.reset_default_graph()
# agent for frequently updating
pg_agent = Policy_Gradiebt_Agent('PG_Agent', num_action)
# init all
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=config)
sess.run(tf.global_variables_initializer())

In [264]:
from IPython.display import Image, display

update_every_episode = 1
print_every_episode = 10
save_video_every_episode = 1000
NUM_EPISODE = 30000
NUM_EXPLORE = 10
NUM_PASS = 20
reward_values = {
    "positive": 1,
    "tick": 0.1,  # reward per timestamp
    "loss": -1,
}
for episode in range(0, NUM_EPISODE + 1):

  # Reset the environment
  game = FlappyBird()
  env = PLE(
      game,
      fps=30,
      display_screen=False,
      reward_values=reward_values,
      rng=np.random.RandomState(1))
  env.reset_game()
  env.act(0)  # dummy input to make sure input screen is correct

  # record frame
  if episode % save_video_every_episode == 0:
    frames = [env.getScreenRGB()]

  # grayscale input screen for this episode
  #input_screens = [preprocess(env.getScreenGrayscale())] * 4
  input_state = [pg_agent.get_state_idx(game.getGameState())] * 4
  # cumulate reward for this episode
  cum_reward = 0

  experiences = []
  t = 0
  while not env.game_over():
    # feed four previous screen, select an action
    action = pg_agent.select_action(input_state[-4:], sess)

    # execute the action and get reward
    reward = env.act(env.getActionSet()[action])

    # record frame
    if episode % save_video_every_episode == 0:
      frames.append(env.getScreenRGB())

    # cumulate reward
    cum_reward += reward

    # append grayscale screen for this episode
    #input_screens.append(preprocess(env.getScreenGrayscale()))
    input_state.append(pg_agent.get_state_idx(game.getGameState()))
    # append experience for this episode
    experiences.append(
        [input_state[-5:-1], action, reward, input_state[-4:]])

    t += 1

  def discount_reward(x, discount_rate):
    discounted_r = np.zeros(len(x))
    num_r = len(x)
    for i in range(num_r):
      discounted_r[i] = x[i] * math.pow(discount_rate, i)
    discounted_r = np.cumsum(discounted_r[::-1])
    return discounted_r[::-1]

  rewards = [e[2] for e in experiences]
  discounted_reward = discount_reward(rewards, pg_agent.discount_factor)

  # normalize
  discounted_reward -= np.mean(discounted_reward)
  discounted_reward /= np.std(discounted_reward)
  train_states = []
  train_actions = []
  train_rewards = []
  train_input_states_plum = []
  for i in range(len(experiences)):
    experiences[i][2] = discounted_reward[i]
    train_states.append(experiences[i][0])
    train_actions.append(experiences[i][1])
    train_rewards.append(experiences[i][2])
    train_input_states_plum.append(experiences[i][3])
  loss = pg_agent.update_policy(train_states, train_actions, train_rewards,
                                train_input_states_plum)

  if episode % print_every_episode == 0 and episode > NUM_EXPLORE:
    print("[{}] time live:{}, cumulated reward: {}, loss: {}".format(
        episode, t, cum_reward, loss))

  if episode % save_video_every_episode == 0 and episode > NUM_EXPLORE:  # for every 5000 episode, record an animation
    clip = make_anim(frames, fps=60, true_image=True).rotate(-90)
    clip.write_videofile("movie/pg_{}.webm".format(episode), fps=60)
    #display(clip.ipython_display(fps=60, autoplay=1, loop=1))

[20] time live:61, cumulated reward: 5.099999999999994, loss: -0.0017613895470276475
[30] time live:61, cumulated reward: 5.099999999999994, loss: 0.0004440057382453233
[40] time live:61, cumulated reward: 5.099999999999994, loss: -0.0012428720947355032
[50] time live:48, cumulated reward: 3.799999999999999, loss: -0.0006696780328638852
[60] time live:46, cumulated reward: 3.5999999999999996, loss: 0.0015977568691596389
[70] time live:41, cumulated reward: 3.1000000000000014, loss: 0.0010510653955861926
[80] time live:53, cumulated reward: 4.299999999999997, loss: 0.0008013383485376835
[90] time live:61, cumulated reward: 5.099999999999994, loss: -5.5000429711071774e-05
[100] time live:61, cumulated reward: 5.099999999999994, loss: -0.0002705933584365994
[110] time live:49, cumulated reward: 3.8999999999999986, loss: 0.00026915024500340223
[120] time live:52, cumulated reward: 4.1999999999999975, loss: 0.0005383308161981404
[130] time live:60, cumulated reward: 4.999999999999995, loss:

[990] time live:54, cumulated reward: 4.399999999999997, loss: 0.001070128520950675
[1000] time live:61, cumulated reward: 5.099999999999994, loss: -0.002358639845624566
[MoviePy] >>>> Building video movie/pg_1000.webm
[MoviePy] Writing video movie/pg_1000.webm


100%|██████████| 63/63 [00:00<00:00, 97.18it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: movie/pg_1000.webm 



[1010] time live:42, cumulated reward: 3.200000000000001, loss: -0.0043884688057005405
[1020] time live:49, cumulated reward: 3.8999999999999986, loss: 0.0018799061654135585
[1030] time live:55, cumulated reward: 4.4999999999999964, loss: -0.0032919971272349358
[1040] time live:47, cumulated reward: 3.6999999999999993, loss: -0.0013284480664879084
[1050] time live:47, cumulated reward: 3.6999999999999993, loss: 0.00038370172842405736
[1060] time live:61, cumulated reward: 5.099999999999994, loss: -0.0013746042968705297
[1070] time live:56, cumulated reward: 4.599999999999996, loss: 0.0004116807540412992
[1080] time live:61, cumulated reward: 5.099999999999994, loss: 0.0016664286376908422
[1090] time live:61, cumulated reward: 5.099999999999994, loss: -0.0012039747089147568
[1100] time live:46, cumulated reward: 3.5999999999999996, loss: 0.004214680753648281
[1110] time live:62, cumulated reward: 5.199999999999994, loss: -0.0015704554971307516
[1120] time live:48, cumulated reward: 3.79

...

[MoviePy] Done.
[MoviePy] >>>> Video ready: movie/pg_29000.webm 



[29010] time live:40, cumulated reward: 3.0000000000000018, loss: -0.05674027279019356
[29020] time live:57, cumulated reward: 4.699999999999996, loss: 0.016977477818727493
[29030] time live:72, cumulated reward: 7.19999999999999, loss: 0.009418540634214878
[29040] time live:51, cumulated reward: 4.099999999999998, loss: 0.009768298827111721
[29050] time live:61, cumulated reward: 5.099999999999994, loss: -0.0028805029578506947
[29060] time live:61, cumulated reward: 5.099999999999994, loss: 0.015320105478167534
[29070] time live:45, cumulated reward: 3.5, loss: 0.0019775815308094025
[29080] time live:60, cumulated reward: 4.999999999999995, loss: -0.030988328158855438
[29090] time live:61, cumulated reward: 5.099999999999994, loss: -0.040673647075891495
[29100] time live:61, cumulated reward: 5.099999999999994, loss: 0.014625079929828644
[29110] time live:55, cumulated reward: 4.4999999999999964, loss: 0.02301434613764286
[29120] time live:51, cumulated reward: 4.099999999999998, loss

[29960] time live:50, cumulated reward: 3.9999999999999982, loss: -0.027254618704319
[29970] time live:57, cumulated reward: 4.699999999999996, loss: -0.0026633278466761112
[29980] time live:64, cumulated reward: 5.399999999999993, loss: -0.020064860582351685
[29990] time live:52, cumulated reward: 4.1999999999999975, loss: -0.016146954149007797
[30000] time live:61, cumulated reward: 5.099999999999994, loss: -0.015461218543350697
[MoviePy] >>>> Building video movie/pg_30000.webm
[MoviePy] Writing video movie/pg_30000.webm


100%|██████████| 63/63 [00:00<00:00, 101.28it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: movie/pg_30000.webm 



#### 最好的結果

In [282]:
from moviepy.editor import *
clip = VideoFileClip("movie/pg_11000.webm")
display(clip.ipython_display(fps=60, autoplay=1, loop=1))

 99%|█████████▊| 69/70 [00:00<00:00, 1025.95it/s]


#### 進入第一個障礙物

In [281]:
from moviepy.editor import *
clip = VideoFileClip("movie/pg_2000.webm")
display(clip.ipython_display(fps=60, autoplay=1, loop=1))

100%|██████████| 71/71 [00:00<00:00, 1110.16it/s]


## Actor-Critic

In [277]:
class Actor_critic:

  def __init__(self, name, num_action, discount_factor=0.99):
    self.exploring_rate = 0.1
    self.discount_factor = discount_factor
    self.num_action = num_action
    self.name = name
    self.bucket_range_per_feature = bucket_range_per_feature
    with tf.variable_scope(name):
      self.build_model()

  def get_state_idx(self, state):
    # instead of using absolute position of pipe, use relative position
    state = copy.deepcopy(state)
    state['next_next_pipe_bottom_y'] -= state['player_y']
    state['next_next_pipe_top_y'] -= state['player_y']
    state['next_pipe_bottom_y'] -= state['player_y']
    state['next_pipe_top_y'] -= state['player_y']

    # sort to make list converted from dict ordered in alphabet order
    state_key = [k for k, v in sorted(state.items())]

    # do bucketing to decrease state space to speed up training
    state_idx = []
    for key in state_key:
        state_idx.append(state[key] / self.bucket_range_per_feature[key])
    return tuple(state_idx)    

  def build_model(self):
    # input: current screen, selected action and reward
    self.input_state = tf.placeholder(
        tf.float32, shape=[None, 8, num_stack])
    self.action = tf.placeholder(tf.int32, [None])
    self.reward = tf.placeholder(tf.float32, [None])
    self.is_training = tf.placeholder(tf.bool, shape=[])

    def value_net(state, reuse=False):
      with tf.variable_scope("fc"):
        for idx in range(num_stack):
          if idx == 0:
            pre_dense = tf.layers.dense(inputs=state[:, :, idx], units=6400, activation=tf.nn.relu, reuse=reuse)
            pre_dense = tf.reshape(pre_dense, [-1, 8, 1])
          else:
            pre_stack = tf.layers.dense(inputs=state[:, :, idx], units=6400, activation=tf.nn.relu, reuse=True)
            pre_stack = tf.reshape(pre_stack, [-1, 8, 1])
            pre_dense = tf.concat([pre_dense, pre_stack], 2) 
            
      with tf.variable_scope(
          "value_net",
          reuse=reuse,
          initializer=tf.truncated_normal_initializer(stddev=1e-2)):
        state_in = tf.reshape(pre_dense, [-1, screen_width, screen_height, num_stack]) 
        conv1 = tf.layers.conv2d(
            inputs=state_in,
            filters=32,
            kernel_size=[8, 8],
            strides=[4, 4],
            padding='SAME',
            activation=tf.nn.relu)
        pool1 = tf.layers.max_pooling2d(
            conv1, pool_size=[2, 2], strides=[2, 2], padding='SAME')

        conv2 = tf.layers.conv2d(
            inputs=pool1,
            filters=64,
            kernel_size=[4, 4],
            strides=[2, 2],
            padding='SAME',
            activation=tf.nn.relu)
        conv3 = tf.layers.conv2d(
            inputs=conv2,
            filters=64,
            kernel_size=[3, 3],
            strides=[1, 1],
            padding='SAME',
            activation=tf.nn.relu)
        flat = tf.contrib.layers.flatten(conv3)
        dense = tf.layers.dense(inputs=flat, units=512, activation=tf.nn.relu)
        V = tf.layers.dense(inputs=dense, units=1, activation=None)
        return V

    def policy_net(state, reuse=False):
      with tf.variable_scope("policy_fc"):
        for idx in range(num_stack):
          if idx == 0:
            pre_dense = tf.layers.dense(inputs=state[:, :, idx], units=6400, activation=tf.nn.relu, reuse=reuse)
            pre_dense = tf.reshape(pre_dense, [-1, 8, 1])
          else:
            pre_stack = tf.layers.dense(inputs=state[:, :, idx], units=6400, activation=tf.nn.relu, reuse=True)
            pre_stack = tf.reshape(pre_stack, [-1, 8, 1])
            pre_dense = tf.concat([pre_dense, pre_stack], 2)
            
      with tf.variable_scope("policy_net", reuse=reuse):
        state_in = tf.reshape(pre_dense, [-1, screen_width, screen_height, num_stack]) 
        conv1 = tf.layers.conv2d(
            inputs=state_in,
            filters=32,
            kernel_size=[8, 8],
            strides=[4, 4],
            padding='SAME',
            activation=tf.nn.relu)
        pool1 = tf.layers.max_pooling2d(
            conv1, pool_size=[2, 2], strides=[2, 2], padding='SAME')

        conv2 = tf.layers.conv2d(
            inputs=pool1,
            filters=64,
            kernel_size=[4, 4],
            strides=[2, 2],
            padding='SAME',
            activation=tf.nn.relu)
        conv3 = tf.layers.conv2d(
            inputs=conv2,
            filters=64,
            kernel_size=[3, 3],
            strides=[1, 1],
            padding='SAME',
            activation=tf.nn.relu)
        self.flat = tf.contrib.layers.flatten(conv3)

        self.dense1 = tf.layers.dense(
            inputs=self.flat, units=512, activation=tf.nn.relu)
        self.dense2 = tf.layers.dense(
            inputs=self.dense1, units=self.num_action, activation=None)
        return self.dense2

    # value
    self.v_output = value_net(
        self.input_state
    )  # Q(s,a,theta) for all a, shape (batch_size, num_action)
    self.tar_V = tf.placeholder(tf.float32, [None])
    self.V_loss = tf.reduce_mean(
        tf.square(self.reward + self.discount_factor * self.tar_V -
                  self.v_output))
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-6)
    g_gvs = optimizer.compute_gradients(
        self.V_loss,
        var_list=[v for v in tf.global_variables() if 'value_net' in v.name])
    self.V_train_op = optimizer.apply_gradients(g_gvs)

    # policy
    self.policy_logit = policy_net(
        self.input_state
    )  # logit of probility(P(s,a,theta)) for all a, shape (batch_size, num_action)
    index = tf.stack([tf.range(tf.shape(self.action)[0]), self.action], axis=1)
    self.prob = tf.gather_nd(
        tf.nn.softmax(self.policy_logit),
        index)  # P(s,a,theta) for selected action, shape (batch_size, 1)

    # loss = E[log(p(s,a))*r]
    self.policy_loss = -tf.reduce_mean(
        tf.log(self.prob + 0.00000001) * self.reward)
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-6)
    g_gvs = optimizer.compute_gradients(
        self.policy_loss,
        var_list=[v for v in tf.global_variables() if 'policy_net' in v.name])
    self.train_op = optimizer.apply_gradients(g_gvs)
    self.pred = tf.multinomial(self.policy_logit,
                               1)  # sample action from distribution

  def select_action(self, input_state, sess):
    input_state = np.array(input_state).transpose([1, 0])
    feed_dict = {
        self.input_state: input_state[None, :],
    }
    action = sess.run(
        self.pred,
        feed_dict=feed_dict)[0][0]  # sameple action from distribution
    return action

  def update_policy(self, input_state, actions, rewards, input_states_plum):
    feed_dict = {
        self.input_state: np.array(input_states_plum).transpose([0, 2, 1]),
    }
    esti_V = sess.run(self.v_output, feed_dict=feed_dict).flatten()
    td_target = rewards + self.discount_factor * esti_V

    feed_dict = {
        self.input_state: np.array(input_state).transpose([0, 2, 1]),
    }
    esti_V = sess.run(self.v_output, feed_dict=feed_dict).flatten()
    td_error = td_target - esti_V
    feed_dict = {
        self.input_state: np.array(input_states_plum).transpose([0, 2, 1]),
    }
    feed_dict = {
        self.input_state: np.array(input_state).transpose([0, 2, 1]),
        self.tar_V: td_target,
        self.reward: rewards,
    }

    V_loss, _ = sess.run([self.V_loss, self.V_train_op], feed_dict=feed_dict)

    feed_dict = {
        self.input_state: np.array(input_state).transpose([0, 2, 1]),
        self.action: actions,
        self.reward: td_error,
    }
    policy_loss, _ = sess.run(
        [self.policy_loss, self.train_op], feed_dict=feed_dict)
    return V_loss, policy_loss

  def update_parameters(self, episode):
    if self.exploring_rate > MIN_EXPLORING_RATE:
      self.exploring_rate -= (0.1 - MIN_EXPLORING_RATE) / 3000000

  def shutdown_explore(self):
    # make action selection greedy
    self.exploring_rate = 0

In [278]:
# init agent
tf.reset_default_graph()
# agent for frequently updating
ac_agent = Actor_critic('AC_Agent', num_action)
# init all
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=config)
sess.run(tf.global_variables_initializer())

In [279]:
from IPython.display import Image, display

update_every_episode = 1
print_every_episode = 10
save_video_every_episode = 1000
NUM_EPISODE = 30000
NUM_EXPLORE = 0
reward_values = {
    "positive": 1,
    "tick": 0.1,  # reward per timestamp
    "loss": -1,
}
for episode in range(0, NUM_EPISODE + 1):

  # Reset the environment
  game = FlappyBird()
  env = PLE(
      game,
      fps=30,
      display_screen=False,
      reward_values=reward_values,
      rng=np.random.RandomState(1))
  env.reset_game()
  env.act(0)  # dummy input to make sure input screen is correct

  # record frame
  if episode % save_video_every_episode == 0:
    frames = [env.getScreenRGB()]

  # grayscale input screen for this episode
  #input_screens = [preprocess(env.getScreenGrayscale())] * 4
  input_state = [ac_agent.get_state_idx(game.getGameState())] * 4
  # cumulate reward for this episode
  cum_reward = 0

  experiences = []
  t = 0
  while not env.game_over():
    # feed four previous screen, select an action
    action = ac_agent.select_action(input_state[-4:], sess)

    # execute the action and get reward
    reward = env.act(env.getActionSet()[action])

    # record frame
    if episode % save_video_every_episode == 0:
      frames.append(env.getScreenRGB())

    # cumulate reward
    cum_reward += reward

    # append grayscale screen for this episode
    #input_screens.append(preprocess(env.getScreenGrayscale()))
    input_state.append(ac_agent.get_state_idx(game.getGameState()))
    # append experience for this episode
    experiences.append(
        [input_state[-5:-1], action, reward, input_state[-4:]])

    t += 1

  def discount_reward(x, discount_rate):
    discounted_r = np.zeros(len(x))
    num_r = len(x)
    for i in range(num_r):
      discounted_r[i] = x[i] * math.pow(discount_rate, i)
    discounted_r = np.cumsum(discounted_r[::-1])
    return discounted_r[::-1]

  rewards = [e[2] for e in experiences]
  discounted_reward = discount_reward(rewards, ac_agent.discount_factor)

  # normalize
  discounted_reward -= np.mean(discounted_reward)
  discounted_reward /= np.std(discounted_reward)
  train_states = []
  train_actions = []
  train_rewards = []
  train_input_states_plum = []
  for i in range(len(experiences)):
    experiences[i][2] = discounted_reward[i]
    train_states.append(experiences[i][0])
    train_actions.append(experiences[i][1])
    train_rewards.append(experiences[i][2])
    train_input_states_plum.append(experiences[i][3])
  loss = ac_agent.update_policy(train_states, train_actions, train_rewards,
                                train_input_states_plum)

  if episode % print_every_episode == 0 and episode > NUM_EXPLORE:
    print("[{}] time live:{}, cumulated reward: {}, loss: {}".format(
        episode, t, cum_reward, loss))

  if episode % save_video_every_episode == 0 and episode > NUM_EXPLORE:  # for every 5000 episode, record an animation
    clip = make_anim(frames, fps=60, true_image=True).rotate(-90)
    clip.write_videofile("movie/ac_{}.webm".format(episode), fps=60)
    #display(clip.ipython_display(fps=60, autoplay=1, loop=1))

[10] time live:58, cumulated reward: 4.799999999999995, loss: (3.9601116, -0.0016315066)
[20] time live:46, cumulated reward: 3.5999999999999996, loss: (3.960111, 0.0053244676)
[30] time live:47, cumulated reward: 3.6999999999999993, loss: (3.9601085, -0.0025144333)
[40] time live:49, cumulated reward: 3.8999999999999986, loss: (3.9601057, -0.0017766174)
[50] time live:65, cumulated reward: 6.499999999999993, loss: (3.9601119, -0.001960461)
[60] time live:55, cumulated reward: 4.4999999999999964, loss: (3.960107, -0.00024115822)
[70] time live:53, cumulated reward: 4.299999999999997, loss: (3.9601045, -0.0003839169)
[80] time live:58, cumulated reward: 4.799999999999995, loss: (3.9601068, -0.0020105098)
[90] time live:45, cumulated reward: 3.5, loss: (3.960104, 5.086263e-06)
[100] time live:61, cumulated reward: 5.099999999999994, loss: (3.9601057, 0.001795972)
[110] time live:41, cumulated reward: 3.1000000000000014, loss: (3.960102, -0.003611588)
[120] time live:61, cumulated reward:

[940] time live:51, cumulated reward: 4.099999999999998, loss: (3.9600987, 0.00012764276)
[950] time live:60, cumulated reward: 4.999999999999995, loss: (3.9601, -0.0055971146)
[960] time live:56, cumulated reward: 4.599999999999996, loss: (3.9601, -0.0012027536)
[970] time live:46, cumulated reward: 3.5999999999999996, loss: (3.9600985, 0.00356782)
[980] time live:61, cumulated reward: 5.099999999999994, loss: (3.960101, 0.0016161809)
[990] time live:46, cumulated reward: 3.5999999999999996, loss: (3.9600995, -0.0008530824)
[1000] time live:55, cumulated reward: 4.4999999999999964, loss: (3.9600997, 0.0027024008)
[MoviePy] >>>> Building video movie/ac_1000.webm
[MoviePy] Writing video movie/ac_1000.webm


 98%|█████████▊| 56/57 [00:00<00:00, 103.25it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: movie/ac_1000.webm 

[1010] time live:35, cumulated reward: 2.500000000000002, loss: (3.9600987, -0.001048599)
[1020] time live:57, cumulated reward: 4.699999999999996, loss: (3.960101, -0.0024415401)
[1030] time live:54, cumulated reward: 4.399999999999997, loss: (3.9601014, 0.0006099277)
[1040] time live:61, cumulated reward: 5.099999999999994, loss: (3.9601007, -0.0026456488)
[1050] time live:46, cumulated reward: 3.5999999999999996, loss: (3.9600995, -0.0010487308)
[1060] time live:54, cumulated reward: 4.399999999999997, loss: (3.9601018, -0.0053539984)
[1070] time live:48, cumulated reward: 3.799999999999999, loss: (3.9601004, -0.0041612387)
[1080] time live:49, cumulated reward: 3.8999999999999986, loss: (3.9600987, 0.0026848656)
[1090] time live:61, cumulated reward: 5.099999999999994, loss: (3.9601011, 0.0036040135)
[1100] time live:49, cumulated reward: 3.8999999999999986, loss: (3.9600985, 0.0036683958)
[1110] time live:49, cumulat

[1920] time live:61, cumulated reward: 5.099999999999994, loss: (3.9600997, 0.003614113)
[1930] time live:52, cumulated reward: 4.1999999999999975, loss: (3.9600997, 0.0060604904)
[1940] time live:53, cumulated reward: 4.299999999999997, loss: (3.9601, 0.0029858283)
[1950] time live:61, cumulated reward: 5.099999999999994, loss: (3.9601007, -0.0054405713)
[1960] time live:61, cumulated reward: 5.099999999999994, loss: (3.9601004, 0.0020656274)
[1970] time live:52, cumulated reward: 4.1999999999999975, loss: (3.9601011, -0.0031021743)
[1980] time live:57, cumulated reward: 4.699999999999996, loss: (3.9600997, -0.00451359)
[1990] time live:49, cumulated reward: 3.8999999999999986, loss: (3.9600997, -0.0047158613)
[2000] time live:54, cumulated reward: 4.399999999999997, loss: (3.9601007, -0.0027956432)
[MoviePy] >>>> Building video movie/ac_2000.webm
[MoviePy] Writing video movie/ac_2000.webm


...

[MoviePy] Done.
[MoviePy] >>>> Video ready: movie/ac_29000.webm 



[29010] time live:54, cumulated reward: 4.399999999999997, loss: (3.9601, 0.02496866)
[29020] time live:41, cumulated reward: 3.1000000000000014, loss: (3.9601007, -0.01854357)
[29030] time live:48, cumulated reward: 3.799999999999999, loss: (3.9600995, -0.013245225)
[29040] time live:47, cumulated reward: 3.6999999999999993, loss: (3.9601002, -0.022911213)
[29050] time live:59, cumulated reward: 4.899999999999995, loss: (3.9601002, 0.015972655)
[29060] time live:45, cumulated reward: 3.5, loss: (3.9600997, -0.018605232)
[29070] time live:72, cumulated reward: 7.19999999999999, loss: (3.9600997, 8.919504e-05)
[29080] time live:55, cumulated reward: 4.4999999999999964, loss: (3.9601002, -0.03912447)
[29090] time live:61, cumulated reward: 5.099999999999994, loss: (3.9600997, -0.039576452)
[29100] time live:56, cumulated reward: 4.599999999999996, loss: (3.9600995, 0.0075675077)
[29110] time live:61, cumulated reward: 5.099999999999994, loss: (3.9601, -0.024106886)
[29120] time live:43, 

[29930] time live:51, cumulated reward: 4.099999999999998, loss: (3.9600995, -0.020904448)
[29940] time live:61, cumulated reward: 5.099999999999994, loss: (3.9600995, -0.032258518)
[29950] time live:59, cumulated reward: 4.899999999999995, loss: (3.9601002, -0.024703203)
[29960] time live:53, cumulated reward: 4.299999999999997, loss: (3.9601, -0.03423982)
[29970] time live:52, cumulated reward: 4.1999999999999975, loss: (3.9600997, 0.09447226)
[29980] time live:61, cumulated reward: 5.099999999999994, loss: (3.9600995, -0.0024897153)
[29990] time live:61, cumulated reward: 5.099999999999994, loss: (3.9600997, -0.039155383)
[30000] time live:57, cumulated reward: 4.699999999999996, loss: (3.9600995, 0.020440152)
[MoviePy] >>>> Building video movie/ac_30000.webm
[MoviePy] Writing video movie/ac_30000.webm


 98%|█████████▊| 58/59 [00:00<00:00, 99.10it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: movie/ac_30000.webm 



#### 最好的結果

In [283]:
from moviepy.editor import *
clip = VideoFileClip("movie/ac_8000.webm")
display(clip.ipython_display(fps=60, autoplay=1, loop=1))

 99%|█████████▊| 78/79 [00:00<00:00, 997.46it/s]


#### 進入第一個障礙物

In [284]:
from moviepy.editor import *
clip = VideoFileClip("movie/ac_3000.webm")
display(clip.ipython_display(fps=60, autoplay=1, loop=1))

100%|██████████| 68/68 [00:00<00:00, 669.79it/s]


# Result

這次的作業我照著講義中的步驟來做，input的部份我用game.gamestate()來抓state，然後preprocessing的部份使用lab16的作法，也有依stack_num將他們stack起來，處理完後model的一開始加了一層dense layer，將state拉成6400 units，再將它reshape成和本來screen一樣的大小（80*80），結果以單純DQN、Policy gradient、Actor-Critic三個部份來做比較。

將input換成state之後，變得更難train，這次我三個部分都train了先30000個episode，train完才發現要訓練到完全收斂要非常長的時間，每1000次存一次影片。

結果的部分我分別放了三個部份只在30000個episode的訓練下最好的結果，以及三個部份能進到第一個障礙物的影片。

單純DQN的作法，在11000個episode時可以進入第一個障礙物，但無法完全通過，一直到22000 episode時可以完全通過第一個障礙物，但沒有辦法很穩定的越來越進步，訓練完30000個episode時最好可以完全通過兩個障礙物，但也不是很穩定。

Policy gradient 的部分，在2000episode和9000episode的時候就可以進入第一個障礙物，比單純DQN快，不過也沒辦法完全通過，在30000episode中還有幾次的結果也可以進入到第一個障礙物，但都沒有辦法完全通過，也都沒辦法穩定的進步，2000episode可以進入第一個障礙物，但一直到9000episode以前都沒辦法在進到第一個障礙物，相當不穩定，可能是跟policy gradient使用stochestic的policy以及容易找到local minimum有關，感覺要訓練到收斂要非常長的時間。

Actor-critic的部分，大概在3000個episode時可以進入到第一個障礙物，比單純DQN快，但也沒辦法完全通過，8000episode時就可以完全通過第一個障礙物，但後來的episode也不是很穩定，不是卡在第一個障礙物裡就是直接撞上第一個障礙物，感覺要收斂也是需要非常長的時間。